In [1]:
def Display(df_list, axis=0, names=None):
    import ipywidgets as widgets
    
    widget_list = [] 
    names = [f'df_{i}' for i in range(0, len(df_list))] if names is None else names
    for n, df in zip(names, df_list):      
        
        widget = widgets.Output() # create output widgets
        with widget:              # render in output widgets
            print(f'{n}:')
            display(df)
        widget_list.append(widget)
        
    # create HBox
    hbox = widgets.HBox(widget_list) if axis else widgets.VBox(widget_list)
    display(hbox)

In [2]:
import pydotplus as ptp
def buildTree(root):
    def _buildTree(graph, node_dict):
        #--------------------------------------
        # Set info nodo
        n = ptp.Node(str(hex(id(node_dict))), shape='box', style='rounded') # id Node -> locazione memoria del dict -> nodo viene riutilizzato
        n.set('label' , str(node_dict['var'])) # Nome che compare nel diagramma
        graph.add_node(n)
        
        # ------------------------
        #'nPath':0, 'avgLenPath':0
        try:
            label  = f"{str(node_dict['var'])}\n" 
            label += f"nPath:{node_dict['nPath']}\n" 
            label += "avgL: {:0.3}".format(node_dict['avgLenPath'])
            n.set('label' , label)
        except:
            pass

        #--------------------------
        if node_dict['isClosed']:
            n.set_color('orange') 
            n.set_style("filled")
                     
        if node_dict['constr']=='true':
            n.set("color", 'green') 
            n.set_style("filled")
        
        # Temporaeno -> Nodo in cui è stata interrotta la ricerca
        if not(node_dict['isClosed']):
            if not(node_dict['children'][0]) and not(node_dict['children'][1]):
                n.set("color", 'blue') 
                n.set_style("filled")
    
        #-------------------------
        # Set add children
        if node_dict['children'][0] : # esiste, Not None
            left = _buildTree(graph, node_dict['children'][0])
            edgeL = ptp.Edge(n, left, label='False')
            graph.add_edge(edgeL)
            
        if node_dict['children'][1] : # esiste, Not None
            right = _buildTree(graph, node_dict['children'][1])
            edgeR = ptp.Edge(n, right, label='True')
            graph.add_edge(edgeR)
            
        return n.get_name() # ritorno id del nodo
    
    #------------------------------------------
    # Set graph 
    g = ptp.Dot()
    g.set_root(_buildTree(g, root))
    return g
        
#-----------------------------       
#gr =  buildTree(treeVars) 
#gr.write_svg('test.svg')

In [3]:
def cut_nodes(node):
    if node['isClosed']:
        node['children'].clear()# elimino dalla memoria
        node['children']=[None, None]
        return
    
    if node['children'][0]: cut_nodes(node['children'][0])
    if node['children'][1]: cut_nodes(node['children'][1])
    return
     
#cut_nodes(treeVars)

In [4]:
from pandas import DataFrame
def saveTree(root, fileName='treeVarsPriority'):
    
    # {"constr": reduced_formula, "var":None, "children":[None, None], "isClosed":False }
    def _saveNode(node):
        nodeList = []
        try:
            newNode = {'idNode':hex(id(node)), 'var': node['var'], "left":None, "right":None, 
                   'isClosed': node['isClosed'], 'nPath':node['nPath'] , 'avgLenPath':node['avgLenPath']}
        except:
            newNode = {'idNode':hex(id(node)), 'var': node['var'], "left":None, "right":None, 'isClosed': node['isClosed']}
        
        if node['var'] is None: 
            return [newNode]
        
        if node['children'][0]: # not None
            newNode['left'] = hex(id(node['children'][0]))
            nodeList += _saveNode(node['children'][0] ) 
           
        if node['children'][1]: # not None
            newNode['right'] = hex(id(node['children'][1]))
            nodeList += _saveNode(node['children'][1] ) 
           
        return [newNode] + nodeList 
    
    #----------------------------------
    df = DataFrame(_saveNode(root))
    mapping = dict( (v,k) for k,v in df['idNode'].to_dict().items() ) | {None:None} # inverto mapping key:values
    df['left']  = df['left'].map(lambda l : str(mapping[l]))
    df['right'] = df['right'].map(lambda r : str(mapping[r]))
    df['idNode'] = df.index 
    df.to_csv(f'{fileName}.csv', index=False)
    
    
    return df

In [5]:
import sys
from lark import Lark, Tree, Token 
def raw_LarkTree_SpaceUsage(tree, sep=' ', tab=0, verbose=False):
    
    # return the number of bytes
    # size of: class initialization, + data + list of children (NOT children, only pointers are considered) 
    
    size = sys.getsizeof(tree) 
    if isinstance(tree, type(Tree('',[])) ):
        size += sys.getsizeof(tree.data) + sys.getsizeof(tree.children)
        print(f'{sep*tab}{tree.data}: {size}') if verbose else None
        
        for c in tree.children:
            size += raw_LarkTree_SpaceUsage(c, sep, tab+1, verbose)
    
    elif isinstance(tree, type(Token('','')) ):
        
        size += sys.getsizeof(tree.value)
        print(f'{sep*tab}{tree.value}: {size}') if verbose else None
    else:
        print('Unknow')
        
    return size

In [6]:
def raw_myTree_SpaceUsage(myTree, sep=' ', tab=0, verbose=False):
    
    # return the number of bytes
    # size of: class initialization, + data + list of children (NOT children, only pointers are considered) 
    
    size = sys.getsizeof(myTree)
    keys = sum(map(sys.getsizeof, myTree.keys())) # None, str, int are counted here, But not other structures
    vals = sum(map(sys.getsizeof, myTree.values()))
    print(f'{sep*tab}var:{myTree["v"]}, size:{size}, keys:{keys}, vals:{vals} -> {size+keys+vals}') if verbose else None
    
    size += keys + vals
    for c in myTree['c']:
        if c is None:
            none = sys.getsizeof(c)
            size += none
            print(f'{sep*tab}none: {none}') if verbose else None
        else:
            res = raw_myTree_SpaceUsage(c, sep, tab+1, verbose)
            size += res
    
    return size

if __name__=='__main__':

    a = {'v':'var_a','c':[None, None], 'v2':None}
    b = {'v':'var_b','c':[None, None], 'v2':None}
    d = {'v':'var_d','c':[a, b], 'v2':None}
    e = {'v':'var_e','c':[None, d], 'v2':None}

    print(raw_myTree_SpaceUsage(e, sep='  ', verbose=True))

var:var_e, size:232, keys:151, vals:142 -> 525
none: 16
  var:var_d, size:232, keys:151, vals:142 -> 525
    var:var_a, size:232, keys:151, vals:142 -> 525
    none: 16
    none: 16
    var:var_b, size:232, keys:151, vals:142 -> 525
    none: 16
    none: 16
2180
